In [1]:
import ROOT
import numpy as np
import fnmatch
from os import listdir

# Load macros using the compiled .so files, notice the change in filename:
ROOT.gSystem.Load('../../macros/./Nhelpers_C.so')
ROOT.gSystem.Load('../../macros/./tdrstyle_mod15_C.so')

style = ROOT.setTDRStyle()
ROOT.gStyle.SetPalette(ROOT.kGreyYellow)  # LightTemperature

ROOT.EnableImplicitMT(3)

Welcome to JupyROOT 6.26/02


In [6]:
# Simulation
# This line finds all files in the directory that end with .root
files1 = list(fnmatch.filter(listdir(
    "../../../data/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/30000"), "*.root"))

# Load the files and create the RDataFrame
chain1 = ROOT.TChain("Events")
for file in files1[0:1]:
    chain1.Add(
        "../../../data/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/30000/"+file)

# Some useful info / confirmation
print(f"Event count (MC): {chain1.GetEntries()}")

df_MC = ROOT.RDataFrame(chain1)

# Data
files2 = list(fnmatch.filter(
    listdir("../../../data/UL2018_MiniAODv2_JMENanoAODv9-v1/40000"), "*.root"))
chain2 = ROOT.TChain("Events")
for file in files2[0:1]:
    chain2.Add("../../../data/UL2018_MiniAODv2_JMENanoAODv9-v1/40000/"+file)
print(f"Event count (DT): {chain2.GetEntries()}")

df_DT = ROOT.RDataFrame(chain2)

Event count (MC): 684000
Event count (DT): 835673


In [3]:
hlt = [40, 60, 80, 140, 200, 260, 320, 400, 450, 500]
offline = [64, 84, 114, 196, 272, 330, 395, 468, 548, 686]
hlt_filt = "false "

for hlt_temp, reco in zip(hlt, offline):
    hlt_filt += f" || ((pTavg > {reco}) && (HLT_PFJet{hlt_temp}))"

In [35]:
df_MC1 = df_MC.Filter("Jet_pt.size() >= 2") \
    .Define("pTavg", "(Jet_pt[0] + Jet_pt[1]) * 0.5") \
    .Filter("(Jet_pt.size() == 2) || ((Jet_pt[2] < 15) || (Jet_pt[2] < 0.3*pTavg))") \
    .Filter("ROOT::VecOps::DeltaPhi(Jet_phi[0], Jet_phi[1]) > 2.8") \
    .Filter("(Jet_pt[0] > 15) && (Jet_pt[1] > 15)") \
    .Filter("(abs(Jet_eta[0]) < 1.3) && (abs(Jet_eta[1]) < 1.3)") \
    .Define("rIdx", "rand_oneZero()") \
    .Redefine("Jet_nConstituents", "Take(Jet_nConstituents, rIdx)") \
    .Redefine("GenJet_nConstituents", "Take(GenJet_nConstituents, rIdx)") \
    .Define("pTtag", "Take(Jet_pt, rIdx)")

df_DT1 = df_DT.Filter("(Jet_pt.size() >= 2)") \
    .Define("pTavg", "(Jet_pt[0] + Jet_pt[1]) * 0.5") \
    .Filter("(Jet_pt.size() == 2) || ((Jet_pt[2] < 15) || (Jet_pt[2] < 0.3*pTavg))") \
    .Filter("ROOT::VecOps::DeltaPhi(Jet_phi[0], Jet_phi[1]) > 2.8") \
    .Filter("(Jet_pt[0] > 15) && (Jet_pt[1] > 15)") \
    .Filter("(abs(Jet_eta[0]) < 1.3) && (abs(Jet_eta[1]) < 1.3)") \
    .Define("rIdx", "rand_oneZero()") \
    .Redefine("Jet_nConstituents", "Take(Jet_nConstituents, rIdx)") \
    .Define("pTtag", "Take(Jet_pt, rIdx)")

data = np.array((1,2,34), dtype=)
#      .Filter(hlt_filt) \

# jou

In [36]:
bins = np.array((1, 5, 6, 8, 10, 12, 15, 18, 21, 24, 28, 32, 37, 43, 49, 56, 64, 74, 84, 97, 114, 133,
                 153, 174, 196, 220, 245, 272, 300, 330, 362, 395, 430, 468, 507, 548, 592, 638, 686, 737,
                 790, 846, 905, 967, 1032, 1101, 1172, 1248, 1327, 1410, 1497, 1588, 1684, 1784, 1890, 2000,
                 2116, 2238, 2366, 2500, 2640, 2787, 2941, 3103, 3273, 3450, 3637, 3832, 4037, 4252, 4477, 4713,
                 4961, 5220, 5492, 5777, 6076, 6389, 6717, 7000), dtype=np.double)

cnt = len(bins)-1

DT_Hist = df_DT1.Histo2D(
    ("dtHist", "dtHist", cnt, bins, 100, 0, 100), "pTtag", "Jet_nConstituents")
MC_recoHist = df_MC1.Histo2D(
    ("mcRecoHist", "mcRecoHist", cnt, bins, 100, 0, 100), "pTtag", "Jet_nConstituents")
MC_genHist = df_MC1.Histo2D(
    ("mcGenHist", "mcGenHist", cnt, bins, 100, 0, 100), "pTtag", "GenJet_nConstituents")

In [37]:
def tdrAxes(hist, y_low, y_up):
    hist.GetYaxis().SetRangeUser(y_low, y_up)
    hist.GetXaxis().SetRangeUser(15, bins[-1])
    hist.GetXaxis().SetMoreLogLabels()
    hist.GetXaxis().SetNoExponent()

canv = ROOT.TCanvas("canv","canv", 800, 600)

dim = int(np.floor(np.sqrt(cnt)))
canv.Divide(dim, dim)
late = ROOT.TLatex()
late.SetNDC()

skip = 7

for i in range(dim*dim):
    c = canv.cd(i+1)

    MC_recoHist.ProjectionY(f"{i+skip}_reco",i+skip,i+skip).Draw("SAME PLC PMC")
    MC_genHist.ProjectionY(f"{i+skip}_gen",i+skip,i+skip).Draw("SAME PLC PMC")
    DT_Hist.ProjectionY(f"{i+skip}_DT",i+skip,i+skip).Draw("SAME PLC PMC")

    tdrAxes(DT_Hist.GetValue(), 0, 100)

    late.DrawLatex(.575, .7, f'{DT_Hist.GetXaxis().GetBinLowEdge(i+skip)} <= pT < {DT_Hist.GetXaxis().GetBinUpEdge(i+skip)}')

canv.Print("plots.pdf")

Info in <TCanvas::Print>: pdf file plots.pdf has been created


In [38]:
%jsroot on
canv = ROOT.TCanvas("canv2", "canv2",800, 600)
canv.Divide(1,3)
canv.cd(1)
DT_Hist.GetValue().Draw("colz")
canv.cd(2)
MC_recoHist.GetValue().Draw("colz")
canv.cd(3)
MC_genHist.GetValue().Draw("colz")
canv.Draw("colz")